In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
submit = True
search = False
estimators = 20
df = pd.read_csv("../input/football-match-probability-prediction/train.csv")
X_test = pd.read_csv("../input/football-match-probability-prediction/test.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:

display(df.head())
df.shape
for i in df.columns:
    print(i)

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,...,away_team_history_league_id_1,away_team_history_league_id_2,away_team_history_league_id_3,away_team_history_league_id_4,away_team_history_league_id_5,away_team_history_league_id_6,away_team_history_league_id_7,away_team_history_league_id_8,away_team_history_league_id_9,away_team_history_league_id_10
0,11906497,away,Newell's Old Boys,River Plate,2019-12-01 00:45:00,Superliga,636,False,468196.0,468200.0,...,1122.0,642.0,636.0,636.0,636.0,1122.0,636.0,642.0,636.0,1122.0
1,11984383,home,Real Estelí,Deportivo Las Sabanas,2019-12-01 01:00:00,Primera Division,752,False,516788.0,22169161.0,...,752.0,752.0,752.0,752.0,752.0,752.0,752.0,752.0,752.0,752.0
2,11983301,draw,UPNFM,Marathón,2019-12-01 01:00:00,Liga Nacional,734,False,2510608.0,456313.0,...,734.0,734.0,734.0,734.0,734.0,734.0,734.0,734.0,734.0,734.0
3,11983471,away,León,Morelia,2019-12-01 01:00:00,Liga MX,743,False,1552508.0,465797.0,...,743.0,743.0,743.0,743.0,743.0,743.0,743.0,743.0,746.0,743.0
4,11883005,home,Cobán Imperial,Iztapa,2019-12-01 01:00:00,Liga Nacional,705,False,429958.0,426870.0,...,705.0,705.0,705.0,705.0,705.0,705.0,705.0,705.0,705.0,705.0


id
target
home_team_name
away_team_name
match_date
league_name
league_id
is_cup
home_team_coach_id
away_team_coach_id
home_team_history_match_date_1
home_team_history_match_date_2
home_team_history_match_date_3
home_team_history_match_date_4
home_team_history_match_date_5
home_team_history_match_date_6
home_team_history_match_date_7
home_team_history_match_date_8
home_team_history_match_date_9
home_team_history_match_date_10
home_team_history_is_play_home_1
home_team_history_is_play_home_2
home_team_history_is_play_home_3
home_team_history_is_play_home_4
home_team_history_is_play_home_5
home_team_history_is_play_home_6
home_team_history_is_play_home_7
home_team_history_is_play_home_8
home_team_history_is_play_home_9
home_team_history_is_play_home_10
home_team_history_is_cup_1
home_team_history_is_cup_2
home_team_history_is_cup_3
home_team_history_is_cup_4
home_team_history_is_cup_5
home_team_history_is_cup_6
home_team_history_is_cup_7
home_team_history_is_cup_8
home_team_history_is_cup

In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110938 entries, 0 to 110937
Columns: 190 entries, id to away_team_history_league_id_10
dtypes: float64(162), int64(2), object(26)
memory usage: 160.8+ MB


In [4]:
def add_streak(df):
    h_t_goal = "home_team_history_goal_"
    h_t_opp_goal = "home_team_history_opponent_goal_"
    a_t_goal = "away_team_history_goal_"
    a_t_opp_goal = "away_team_history_opponent_goal_"
    df["away_streak"] = np.nan
    df["home_streak"] = np.nan
    def apply_streaks(x):
        home_streak = 0
        away_streak = 0
        home_streak_over = False
        away_streak_over = False
        
        if  x[h_t_goal+str(1)] > x[h_t_opp_goal+str(1)]:
            home_win_streak = True
        else:
            home_win_streak = False
            
        if  x[a_t_goal+str(1)] > x[a_t_opp_goal+str(1)]:
            away_win_streak = True
        else:
            away_win_streak = False
            
        def check_streak(streak,streak_over,win_streak,team_goal,opp_goal):
            if not streak_over and win_streak and team_goal > opp_goal:
                streak+=1
            elif not streak_over and not win_streak and team_goal < opp_goal:
                streak-=1
            elif not streak_over and team_goal == opp_goal:
                pass
            else:
                streak_over = True
            return streak, streak_over
                
        for i in range(1,11):
            if not home_streak_over:
                home_streak, home_streak_over = check_streak(home_streak,home_streak_over, home_win_streak, x[h_t_goal+str(i)],x[h_t_opp_goal+str(i)])
            if not away_streak_over:
                away_streak, away_streak_over = check_streak(away_streak,away_streak_over, away_win_streak, x[a_t_goal+str(i)],x[a_t_opp_goal+str(i)])
            
        return pd.Series([home_streak,away_streak],index=['home_streak','away_streak'])

    
    df[['home_streak','away_streak']] = df.apply(lambda x: apply_streaks(x), axis=1)
    return df

In [5]:
import category_encoders as ce
from sklearn.model_selection import train_test_split


X_train = df.drop(['target'], axis=1)
y_train = df['target']

X_train = add_streak(X_train)
X_test = add_streak(X_test)

if not submit:
    X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

encoder = ce.OrdinalEncoder(cols=['home_team_name' , 'away_team_name', 'is_cup'])
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)
temp_df = X_test
for i in X_train.columns:
    if X_train[i].dtype == 'object':
        X_train = X_train.drop([i],axis=1)
        X_test = X_test.drop([i],axis=1)
    if "is_cup" in i or "history_is_play_home" in i:
        try:
            X_train = X_train.drop([i],axis=1)
            X_test = X_test.drop([i],axis=1)
        except:
            continue

for i in X_train.columns:
    if 'rating' not in i and 'goal':
        X_train[i] = X_train[i].fillna(-1)
        X_test[i] = X_test[i].fillna(-1)
    else:
        X_train[i] = X_train[i].fillna(X_train[i].mode())
        X_test[i] = X_test[i].fillna(X_train[i].mode())
lag_size = 9
#for i in X_train.columns:
#    if sum([1 for num in range(lag_size,11)  if str(num) in i]) >= 1:
#        X_train = X_train.drop([i],axis=1)
#        X_test = X_test.drop([i],axis=1)

In [6]:
#X_train.head(25)
"""X_train[['home_streak','away_streak']].head(20)
h_t_goal = "away_team_history_goal_"
h_t_opp_goal = "away_team_history_opponent_goal_"
cols = []
cols.append('away_streak')
for i in range(1,5):
    cols.append(h_t_goal+str(i))
    cols.append(h_t_opp_goal+str(i))
X_train[cols].head(20)"""

for i in X_train.columns:
    print(i)

id
home_team_name
away_team_name
league_id
home_team_coach_id
away_team_coach_id
home_team_history_goal_1
home_team_history_goal_2
home_team_history_goal_3
home_team_history_goal_4
home_team_history_goal_5
home_team_history_goal_6
home_team_history_goal_7
home_team_history_goal_8
home_team_history_goal_9
home_team_history_goal_10
home_team_history_opponent_goal_1
home_team_history_opponent_goal_2
home_team_history_opponent_goal_3
home_team_history_opponent_goal_4
home_team_history_opponent_goal_5
home_team_history_opponent_goal_6
home_team_history_opponent_goal_7
home_team_history_opponent_goal_8
home_team_history_opponent_goal_9
home_team_history_opponent_goal_10
home_team_history_rating_1
home_team_history_rating_2
home_team_history_rating_3
home_team_history_rating_4
home_team_history_rating_5
home_team_history_rating_6
home_team_history_rating_7
home_team_history_rating_8
home_team_history_rating_9
home_team_history_rating_10
home_team_history_opponent_rating_1
home_team_history_op

In [7]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss,accuracy_score
if not search:
    best_hyperparams = {'colsample_bytree': 0.9080289013373745, 'gamma': 1.2419885617337658, 'max_depth': 8.0, 'min_child_weight': 4.0, 'n_estimators': 238.0, 'reg_alpha': 41.0, 'reg_lambda': 0.05050398016408508}
if not submit and not search:
    rfc = XGBClassifier(n_estimators=int(best_hyperparams['n_estimators']), max_depth = int(best_hyperparams['max_depth']), gamma = best_hyperparams['gamma'],
                    reg_alpha = int(best_hyperparams['reg_alpha']),min_child_weight=int(best_hyperparams['min_child_weight']),
                    colsample_bytree=int(best_hyperparams['colsample_bytree']))
    #fc = XGBClassifier(n_estimators=estimators)
    rfc.fit(X_train, y_train)
    y_pred = rfc.predict_proba(X_test)
    y_pred_acc = rfc.predict(X_test)

    print(y_test.values)
    print(y_pred)
    print('Log loss score with  : {0:0.4f}'. format(log_loss(y_test.values,y_pred)))
    print('Model accuracy score with : {0:0.4f}'. format(accuracy_score(y_test, y_pred_acc)))

In [8]:
if not submit and not search:
    feature_scores = pd.Series(rfc.feature_importances_, index=X_train.columns).sort_values(ascending=False)

    for i,v in feature_scores.items():
        print(i, v)

In [9]:
#https://www.kaggle.com/prashant111/a-guide-on-xgboost-hyperparameters-tuning
from hyperopt import hp
import xgboost as xgb
from sklearn.metrics import accuracy_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

if search:

    space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
            'gamma': hp.uniform ('gamma', 1,9),
            'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
            'reg_lambda' : hp.uniform('reg_lambda', 0,1),
            'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
            'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
            'n_estimators': hp.quniform('n_estimators', 150, 250, 1),
            'seed': 0
        }

    def objective(space):
        clf=xgb.XGBClassifier(
                        n_estimators =int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
                        reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                        colsample_bytree=int(space['colsample_bytree']))

        evaluation = [( X_train, y_train), ( X_test, y_test)]

        clf.fit(X_train, y_train,
                eval_set=evaluation, eval_metric="mlogloss",
                early_stopping_rounds=100,verbose=False)


        y_pred = clf.predict_proba(X_test)
        accuracy = log_loss(y_test.values,y_pred)
        print ("SCORE:", accuracy)
        return {'loss': accuracy, 'status': STATUS_OK }

    trials = Trials()

    best_hyperparams = fmin(fn = objective,
                            space = space,
                            algo = tpe.suggest,
                            max_evals = 100,
                            trials = trials)

In [10]:
print(best_hyperparams)

{'colsample_bytree': 0.9080289013373745, 'gamma': 1.2419885617337658, 'max_depth': 8.0, 'min_child_weight': 4.0, 'n_estimators': 238.0, 'reg_alpha': 41.0, 'reg_lambda': 0.05050398016408508}


In [11]:
"""from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
n_estimators = range(50, 400, 50)

# grid search
model = XGBClassifier()
param_grid = dict(n_estimators=n_estimators)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_["mean_test_score"]
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]
print()
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
"""

'from sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import StratifiedKFold\nkfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)\nn_estimators = range(50, 400, 50)\n\n# grid search\nmodel = XGBClassifier()\nparam_grid = dict(n_estimators=n_estimators)\ngrid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)\ngrid_result = grid_search.fit(X_train, y_train)\nprint("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))\nmeans = grid_result.cv_results_["mean_test_score"]\nstds = grid_result.cv_results_["std_test_score"]\nparams = grid_result.cv_results_["params"]\nprint()\nfor mean, stdev, param in zip(means, stds, params):\n    print("%f (%f) with: %r" % (mean, stdev, param))\n'

In [12]:
from sklearn.pipeline import Pipeline
if submit:
    pipeline = Pipeline(steps=[
        ("classifier", XGBClassifier(n_estimators=int(best_hyperparams['n_estimators']), max_depth = int(best_hyperparams['max_depth']), gamma = best_hyperparams['gamma'],
                    reg_alpha = int(best_hyperparams['reg_alpha']),min_child_weight=int(best_hyperparams['min_child_weight']),
                    colsample_bytree=int(best_hyperparams['colsample_bytree'])))])

    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict_proba(X_test)
    submission = pd.DataFrame(predictions ,columns=pipeline.classes_)

    #do not forget the id column
    submission['id'] = temp_df['id']
    """for i in ['home','away','draw']:
        #submission.iloc[submission[[i]].max(axis=1)>0.5, i]
        submission.loc[(submission[['home','away','draw']].max(axis=1)>0.5) & (submission[['home','away','draw']].idxmax(axis=1).values == i),i] = 1
        submission.loc[(submission[['home','away','draw']].max(axis=1)>0.5) & (submission[['home','away','draw']].idxmax(axis=1).values != i),i] = 0"""
    display(submission)
    #submit!
    submission.to_csv('submission.csv', index=False)


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:04:03] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,away,draw,home,id
0,0.251757,0.290676,0.457567,17761448
1,0.354338,0.311572,0.334090,17695487
2,0.302682,0.329535,0.367783,17715496
3,0.563898,0.268094,0.168008,17715493
4,0.208940,0.290046,0.501014,17715492
...,...,...,...,...
72706,0.271696,0.319361,0.408943,18450246
72707,0.191018,0.286661,0.522321,18164889
72708,0.486966,0.230916,0.282118,18449018
72709,0.187844,0.270600,0.541556,17958831
